# Create Agent using Ollama hosted Model

In [1]:
import os
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

In [2]:
# Tools supported models https://ollama.com/search?c=tools
# ollama run qwen3:1.7b
# ollama run llama3.2
os.environ["OLLAMA_API_BASE"]="http://localhost:11434"

In [3]:
def get_weather(city: str) -> dict:
    print(f"--- Tool: get_weather called for city: {city} ---")
    city_normalized = city.lower().replace(" ", "")
    mock_weather_db = {
        "newyork": {"status": "success", "report": "The weather in New York is sunny with a temperature of 25°C."},
        "london": {"status": "success", "report": "It's cloudy in London with a temperature of 15°C."},
        "tokyo": {"status": "success", "report": "Tokyo is experiencing light rain and a temperature of 18°C."},
    }
    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"Sorry, I don't have weather information for '{city}'."}

In [4]:
print(get_weather("New York"))
print(get_weather("Paris"))

--- Tool: get_weather called for city: New York ---
{'status': 'success', 'report': 'The weather in New York is sunny with a temperature of 25°C.'}
--- Tool: get_weather called for city: Paris ---
{'status': 'error', 'error_message': "Sorry, I don't have weather information for 'Paris'."}


In [5]:
Ollama_Model = "ollama_chat/llama3.2" # or qwen3:1.7b

In [6]:
weather_agent = Agent(
    name="weather_agent_v1",
    model=LiteLlm(model=Ollama_Model),
    description="Provides weather information for specific cities.",
    instruction="You are a helpful weather assistant. "
                "When the user asks for the weather in a specific city, "
                "use the 'get_weather' tool to find the information. "
                "If the tool returns an error, inform the user politely. "
                "If the tool is successful, present the weather report clearly.",
    tools=[get_weather], # Pass the function directly
)

print(f"Agent '{weather_agent.name}' created using model {Ollama_Model}.")

Agent 'weather_agent_v1' created using model ollama_chat/llama3.2.


In [7]:
session_service = InMemorySessionService()
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

Session created: App='weather_tutorial_app', User='user_1', Session='session_001'


In [8]:
runner = Runner(
    agent=weather_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)
print(f"Runner created for agent '{runner.agent.name}'.")

Runner created for agent 'weather_agent_v1'.


In [9]:
async def call_agent_async(query: str, runner_service:Runner, user_id:str, session_id:str) ->str:
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")
  content = types.Content(role='user', parts=[types.Part(text=query)])
  final_response_text = "Agent did not produce a final response." # Default
  async for event in runner_service.run_async(user_id=user_id, session_id=session_id, new_message=content):
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")
      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found
  return final_response_text

In [10]:
response= await call_agent_async("What is the weather like in London?",
                                   runner_service=runner,
                                   user_id=USER_ID,
                                   session_id=SESSION_ID)

print(response)


>>> User Query: What is the weather like in London?
  [Event] Author: weather_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_call=FunctionCall(
    args={
      'city': 'London'
    },
    id='65bc94f1-2265-4588-b8ca-f9bff3f261dc',
    name='get_weather'
  )
)] role='model'
--- Tool: get_weather called for city: London ---
  [Event] Author: weather_agent_v1, Type: Event, Final: False, Content: parts=[Part(
  function_response=FunctionResponse(
    id='65bc94f1-2265-4588-b8ca-f9bff3f261dc',
    name='get_weather',
    response={
      'report': "It's cloudy in London with a temperature of 15°C.",
      'status': 'success'
    }
  )
)] role='user'
  [Event] Author: weather_agent_v1, Type: Event, Final: True, Content: parts=[Part(
  text='The current weather in London is cloudy with a temperature of 15°C.'
)] role='model'
The current weather in London is cloudy with a temperature of 15°C.
